Raters dataset for analysis

In [53]:
import pandas as pd
import numpy as np
from nltk import agreement

In [54]:
# 1. import files and create all-in-one dataframe


filenames = [
    "Credit",
    "Direct_payment",
    "Fine",
    "Supplies",
    "Tax_deduction",
    "Technical_assistance"
]

raters = [
    "Rater_1",
    "Rater_2",
    "Rater_3"
]

df_rater=pd.DataFrame()
df_all=pd.DataFrame()

# Downloading the csv files in /data
for filename in filenames:
    for rater in raters:
        df = pd.read_excel('data/' + rater + '/Unique_sentence_IDs_' + filename + '.xlsx', skiprows=1, index_col ="Sentence_Id")
#        df2=df.rename(columns = {"Unnamed: 8" : "Other_instrument_2" }) 
        
        df["Tag"]= filename
        df["Rater"] = rater
        
        df_rater = pd.concat([df_rater, df],axis=0)
        
    df_all = pd.concat([df_all, df_rater],axis=0)   
    

    


#    df2=df.rename(columns = {"Is_policy": filename }) 
   


In [55]:
#check whether read OK
df

,original_rank,Similarity _score,Sentence,Is_policy,Is_incentive,Other_instrument,Comments,Unnamed: 8,Tag,Rater
Sentence_Id,,,,,,,,,,
08645ac_125,2,0.855792,- Disponer de una contraparte técnica que será...,0.0,0.0,NaN,NaN,Credit,Technical_assistance,Rater_3
2bb3805_288,3,0.816468,Elaboración y Presentación de el o los Proyect...,0.0,0.0,NaN,NaN,Direct payment,Technical_assistance,Rater_3
eb2af8d_43,14,0.814378,e) velar por el buen funcionamiento y operacio...,0.0,0.0,NaN,NaN,Fine,Technical_assistance,Rater_3
b49a8e7_13,6,0.814056,"b) Disponer de la infraestructura, equipos, in...",0.0,0.0,NaN,NaN,Supplies,Technical_assistance,Rater_3
ce638af_26,21,0.806994,Especialidad: práctica o grupo de prácticas so...,0.0,0.0,NaN,NaN,Tax deduction,Technical_assistance,Rater_3
...,...,...,...,...,...,...,...,...,...,...
3434b51_673,192,0.620186,"- Adicionalmente, la nueva infraestructura con...",NaN,NaN,NaN,NaN,NaN,Technical_assistance,Rater_3
ed1f15e_151,193,0.620007,Los propietarios deberán conservar los diferen...,NaN,NaN,NaN,NaN,NaN,Technical_assistance,Rater_3
632b91b_10,194,0.619817,Que este trabajo técnico compartido de los doc...,NaN,NaN,NaN,NaN,NaN,Technical_assistance,Rater_3


In [56]:
#df2
#df_rater
df_all

,original_rank,Similarity _score,Sentence,Is_policy,Is_incentive,Other_instrument,Comments,Unnamed: 8,Tag,Rater
Sentence_Id,,,,,,,,,,
43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance,NaN,Credit,Credit,Rater_1
12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,NaN,"Describes incentives, but not exactly an incen...",Direct payment,Credit,Rater_1
26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Technical assistance,NaN,Fine,Credit,Rater_1
27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,NaN,Missing context,Supplies,Credit,Rater_1
15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN,NaN,Tax deduction,Credit,Rater_1
...,...,...,...,...,...,...,...,...,...,...
3434b51_673,192,0.620186,"- Adicionalmente, la nueva infraestructura con...",NaN,NaN,NaN,NaN,NaN,Technical_assistance,Rater_3
ed1f15e_151,193,0.620007,Los propietarios deberán conservar los diferen...,NaN,NaN,NaN,NaN,NaN,Technical_assistance,Rater_3
632b91b_10,194,0.619817,Que este trabajo técnico compartido de los doc...,NaN,NaN,NaN,NaN,NaN,Technical_assistance,Rater_3


In [57]:
df_all.to_csv(r'data\raters_dataset.csv')

In [58]:
df_rater.to_csv(r'data\rater3_dataset.csv')

In [59]:
# 2. with raters alongside


filenames = [
    "Credit",
    "Direct_payment",
    "Fine",
    "Supplies",
    "Tax_deduction",
    "Technical_assistance"
]

raters = [
    "Rater_1",
    "Rater_2",
    "Rater_3"
]

df_rater=pd.DataFrame()
df_all_raters=pd.DataFrame()

# Downloading the csv files in /data
for filename in filenames:

    df_rater=pd.DataFrame()
    
    for rater in raters:
        df = pd.read_excel('data/' + rater + '/Unique_sentence_IDs_' + filename + '.xlsx', skiprows=1, index_col ="Sentence_Id")
        df2=df.rename(columns = {"Is_policy" : rater + '_policy_tag' , "Is_incentive" : rater + '_incentive_tag',\
                                 "Other_instrument" : rater + '_other_instrument' , "original_rank" : rater + '_original_rank',\
                                "Similarity _score" : rater + '_similarity_score', "Sentence" : rater + '_sentence'}) 
        df3=df2.drop(["Comments", "Unnamed: 8"], axis = 1)


        df_rater = pd.concat([df_rater, df3],axis=1)
        df_rater["Tag"]= filename
        
    df_all_raters = pd.concat([df_all_raters, df_rater],axis=0)   





    #    df2=df.rename(columns = {"Is_policy": filename }) 

    


    


In [60]:
#df_rater
df_all_raters

,Rater_1_original_rank,Rater_1_similarity_score,Rater_1_sentence,Rater_1_policy_tag,Rater_1_incentive_tag,Rater_1_other_instrument,Tag,Rater_2_original_rank,Rater_2_similarity_score,Rater_2_sentence,Rater_2_policy_tag,Rater_2_incentive_tag,Rater_2_other_instrument,Rater_3_original_rank,Rater_3_similarity_score,Rater_3_sentence,Rater_3_policy_tag,Rater_3_incentive_tag,Rater_3_other_instrument
43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance,Credit,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance,1.0,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance
12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,NaN,Credit,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,NaN,2.0,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,1.0,NaN
26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Technical assistance,Credit,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Supplies,3.0,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Supplies
27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,NaN,Credit,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,NaN,4.0,0.720910,Destinados a cubrir las necesidades productiva...,0.0,1.0,NaN
15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN,Credit,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN,1.0,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434b51_673,192,0.620186,"- Adicionalmente, la nueva infraestructura con...",NaN,NaN,NaN,Technical_assistance,192,0.620186,"- Adicionalmente, la nueva infraestructura con...",NaN,NaN,NaN,192.0,0.620186,"- Adicionalmente, la nueva infraestructura con...",NaN,NaN,NaN
ed1f15e_151,193,0.620007,Los propietarios deberán conservar los diferen...,NaN,NaN,NaN,Technical_assistance,193,0.620007,Los propietarios deberán conservar los diferen...,NaN,NaN,NaN,193.0,0.620007,Los propietarios deberán conservar los diferen...,NaN,NaN,NaN
632b91b_10,194,0.619817,Que este trabajo técnico compartido de los doc...,NaN,NaN,NaN,Technical_assistance,194,0.619817,Que este trabajo técnico compartido de los doc...,NaN,NaN,NaN,194.0,0.619817,Que este trabajo técnico compartido de los doc...,NaN,NaN,NaN
20182ce_62,195,0.619770,Esta nueva inspección se llevará a cabo siempr...,NaN,NaN,NaN,Technical_assistance,195,0.619770,Esta nueva inspección se llevará a cabo siempr...,NaN,NaN,NaN,195.0,0.619770,Esta nueva inspección se llevará a cabo siempr...,NaN,NaN,NaN


In [61]:


df_all_raters.drop(["Rater_2_original_rank","Rater_2_similarity_score", "Rater_2_sentence",\
                    "Rater_3_original_rank","Rater_3_similarity_score", "Rater_3_sentence"], axis = 1, inplace=True)

In [62]:
#export now to see whether removal works ok

df_all_raters.to_csv(r'data\pre_raters_dataset.csv')

In [63]:
#removed all NaN rows - the only non-NaN are the labels
df_all_raters.dropna(axis=0, thresh=5, subset=None, inplace=True)
df_all_raters

,Rater_1_original_rank,Rater_1_similarity_score,Rater_1_sentence,Rater_1_policy_tag,Rater_1_incentive_tag,Rater_1_other_instrument,Tag,Rater_2_policy_tag,Rater_2_incentive_tag,Rater_2_other_instrument,Rater_3_policy_tag,Rater_3_incentive_tag,Rater_3_other_instrument
43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance,Credit,0.0,1.0,Technical assistance,0.0,1.0,Technical assistance
12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,NaN,Credit,0.0,0.0,NaN,0.0,1.0,NaN
26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Technical assistance,Credit,0.0,1.0,Supplies,0.0,1.0,Supplies
27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,NaN,Credit,0.0,0.0,NaN,0.0,1.0,NaN
15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN,Credit,1.0,1.0,NaN,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
f1ebd15_30,26,0.678436,"Para estos efectos, ""la Corporación"" deberá fo...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,0.0,0.0,NaN
daebb8c_7,152,0.678430,Que las acciones y obras que componen el refer...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,0.0,0.0,NaN
2bb3805_424,153,0.678387,En estos casos corresponderá al SERVIU el desa...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,1.0,1.0,NaN
7cffd6b_159,154,0.678337,"e) Módulo de Operación y Explotación, que perm...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,0.0,0.0,NaN


In [64]:
#count the valid values 
df_all_raters.count(0)

Rater_1_original_rank       3089
Rater_1_similarity_score    3089
Rater_1_sentence            3089
Rater_1_policy_tag           600
Rater_1_incentive_tag        600
Rater_1_other_instrument      66
Tag                         3089
Rater_2_policy_tag          2563
Rater_2_incentive_tag       2563
Rater_2_other_instrument     229
Rater_3_policy_tag          2686
Rater_3_incentive_tag       2686
Rater_3_other_instrument     253
dtype: int64

In [65]:
#count the valid values 
df_all_raters.count(1)

43f0373_23     13
12a91a7_159    10
26b00aa_103    13
27d89cb_179    10
15df6c5_144    10
               ..
f1ebd15_30      6
daebb8c_7       6
2bb3805_424     6
7cffd6b_159     6
db2046d_42      6
Length: 3089, dtype: int64

In [66]:

df_all_raters["Valid_policy_tags"]=df_all_raters[["Rater_1_policy_tag","Rater_2_policy_tag","Rater_3_policy_tag"]].count(1)
df_all_raters["Valid_incentive_tags"]=df_all_raters[["Rater_1_incentive_tag","Rater_2_incentive_tag","Rater_3_incentive_tag"]].count(1)
df_all_raters["Pos_policy_tags"]=df_all_raters[["Rater_1_policy_tag","Rater_2_policy_tag","Rater_3_policy_tag"]].sum(1)
df_all_raters["Pos_incentive_tags"]=df_all_raters[["Rater_1_incentive_tag","Rater_2_incentive_tag","Rater_3_incentive_tag"]].sum(1)
df_all_raters["Alternative_tags"]=df_all_raters[["Rater_1_other_instrument","Rater_2_other_instrument","Rater_3_other_instrument"]].count(1)
df_all_raters

,Rater_1_original_rank,Rater_1_similarity_score,Rater_1_sentence,Rater_1_policy_tag,Rater_1_incentive_tag,Rater_1_other_instrument,Tag,Rater_2_policy_tag,Rater_2_incentive_tag,Rater_2_other_instrument,Rater_3_policy_tag,Rater_3_incentive_tag,Rater_3_other_instrument,Valid_policy_tags,Valid_incentive_tags,Pos_policy_tags,Pos_incentive_tags,Alternative_tags
43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance,Credit,0.0,1.0,Technical assistance,0.0,1.0,Technical assistance,3,3,0.0,3.0,3
12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,NaN,Credit,0.0,0.0,NaN,0.0,1.0,NaN,3,3,0.0,1.0,0
26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Technical assistance,Credit,0.0,1.0,Supplies,0.0,1.0,Supplies,3,3,0.0,3.0,3
27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,NaN,Credit,0.0,0.0,NaN,0.0,1.0,NaN,3,3,0.0,1.0,0
15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN,Credit,1.0,1.0,NaN,1.0,1.0,NaN,3,3,3.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f1ebd15_30,26,0.678436,"Para estos efectos, ""la Corporación"" deberá fo...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,0.0,0.0,NaN,1,1,0.0,0.0,0
daebb8c_7,152,0.678430,Que las acciones y obras que componen el refer...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,0.0,0.0,NaN,1,1,0.0,0.0,0
2bb3805_424,153,0.678387,En estos casos corresponderá al SERVIU el desa...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,1.0,1.0,NaN,1,1,1.0,1.0,0
7cffd6b_159,154,0.678337,"e) Módulo de Operación y Explotación, que perm...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,0.0,0.0,NaN,1,1,0.0,0.0,0


In [67]:

df_all_raters["Incentive_tag_majority"] = np.where((df_all_raters["Pos_incentive_tags"]/df_all_raters["Valid_incentive_tags"])>=0.5, 1, 0)
df_all_raters["Incentive_tag_any"] = np.where((df_all_raters["Pos_incentive_tags"])>=1, 1, 0) 
df_all_raters["No_incentive_tag"] = np.where((df_all_raters["Pos_incentive_tags"])==0, 1, 0) 
df_all_raters["Policy_tag_majority"] = np.where((df_all_raters["Pos_policy_tags"]/df_all_raters["Valid_policy_tags"])>=0.5, 1, 0)
df_all_raters["Policy_tag_any"] = np.where((df_all_raters["Pos_policy_tags"])>=1, 1, 0) 
df_all_raters["No_policy_tag"] = np.where((df_all_raters["Pos_policy_tags"])==0, 1, 0) 
df_all_raters["Incentive_uncategorised"] = df_all_raters["Incentive_tag_any"]-df_all_raters["Policy_tag_any"] 
df_all_raters

,Rater_1_original_rank,Rater_1_similarity_score,Rater_1_sentence,Rater_1_policy_tag,Rater_1_incentive_tag,Rater_1_other_instrument,Tag,Rater_2_policy_tag,Rater_2_incentive_tag,Rater_2_other_instrument,...,Pos_policy_tags,Pos_incentive_tags,Alternative_tags,Incentive_tag_majority,Incentive_tag_any,No_incentive_tag,Policy_tag_majority,Policy_tag_any,No_policy_tag,Incentive_uncategorised
43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance,Credit,0.0,1.0,Technical assistance,...,0.0,3.0,3,1,1,0,0,0,1,1
12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,NaN,Credit,0.0,0.0,NaN,...,0.0,1.0,0,0,1,0,0,0,1,1
26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Technical assistance,Credit,0.0,1.0,Supplies,...,0.0,3.0,3,1,1,0,0,0,1,1
27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,NaN,Credit,0.0,0.0,NaN,...,0.0,1.0,0,0,1,0,0,0,1,1
15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN,Credit,1.0,1.0,NaN,...,3.0,3.0,0,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f1ebd15_30,26,0.678436,"Para estos efectos, ""la Corporación"" deberá fo...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,...,0.0,0.0,0,0,0,1,0,0,1,0
daebb8c_7,152,0.678430,Que las acciones y obras que componen el refer...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,...,0.0,0.0,0,0,0,1,0,0,1,0
2bb3805_424,153,0.678387,En estos casos corresponderá al SERVIU el desa...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,...,1.0,1.0,0,1,1,0,1,1,0,0
7cffd6b_159,154,0.678337,"e) Módulo de Operación y Explotación, que perm...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,NaN,...,0.0,0.0,0,0,0,1,0,0,1,0


In [68]:
df_all_raters[df_all_raters == 1].sum(axis=1)

43f0373_23      8.0
12a91a7_159     5.0
26b00aa_103     7.0
27d89cb_179     5.0
15df6c5_144    11.0
               ... 
f1ebd15_30      4.0
daebb8c_7       4.0
2bb3805_424    10.0
7cffd6b_159     4.0
db2046d_42      4.0
Length: 3089, dtype: float64

In [69]:
df_all_raters.dtypes

Rater_1_original_rank         int64
Rater_1_similarity_score    float64
Rater_1_sentence             object
Rater_1_policy_tag          float64
Rater_1_incentive_tag       float64
Rater_1_other_instrument     object
Tag                          object
Rater_2_policy_tag          float64
Rater_2_incentive_tag       float64
Rater_2_other_instrument     object
Rater_3_policy_tag          float64
Rater_3_incentive_tag       float64
Rater_3_other_instrument     object
Valid_policy_tags             int64
Valid_incentive_tags          int64
Pos_policy_tags             float64
Pos_incentive_tags          float64
Alternative_tags              int64
Incentive_tag_majority        int32
Incentive_tag_any             int32
No_incentive_tag              int32
Policy_tag_majority           int32
Policy_tag_any                int32
No_policy_tag                 int32
Incentive_uncategorised       int32
dtype: object

In [70]:
df_all_raters.to_csv(r'data\raters_column_dataset.csv')

In [71]:
#split the document part from the reference 
df_doc=df_all_raters.reset_index()
refs=df_doc['index'].str.split('_', n=1, expand=True)
df_doc['doc_ref']=refs[0]
df_doc['sentence_ref']=refs[1]
df_doc
#does the index need to be reset?

,index,Rater_1_original_rank,Rater_1_similarity_score,Rater_1_sentence,Rater_1_policy_tag,Rater_1_incentive_tag,Rater_1_other_instrument,Tag,Rater_2_policy_tag,Rater_2_incentive_tag,...,Alternative_tags,Incentive_tag_majority,Incentive_tag_any,No_incentive_tag,Policy_tag_majority,Policy_tag_any,No_policy_tag,Incentive_uncategorised,doc_ref,sentence_ref
0,43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Technical assistance,Credit,0.0,1.0,...,3,1,1,0,0,0,1,1,43f0373,23
1,12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,NaN,Credit,0.0,0.0,...,0,0,1,0,0,0,1,1,12a91a7,159
2,26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Technical assistance,Credit,0.0,1.0,...,3,1,1,0,0,0,1,1,26b00aa,103
3,27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,NaN,Credit,0.0,0.0,...,0,0,1,0,0,0,1,1,27d89cb,179
4,15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,NaN,Credit,1.0,1.0,...,0,1,1,0,1,1,0,0,15df6c5,144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,f1ebd15_30,26,0.678436,"Para estos efectos, ""la Corporación"" deberá fo...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,...,0,0,0,1,0,0,1,0,f1ebd15,30
3085,daebb8c_7,152,0.678430,Que las acciones y obras que componen el refer...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,...,0,0,0,1,0,0,1,0,daebb8c,7
3086,2bb3805_424,153,0.678387,En estos casos corresponderá al SERVIU el desa...,NaN,NaN,NaN,Technical_assistance,NaN,NaN,...,0,1,1,0,1,1,0,0,2bb3805,424
3087,7cffd6b_159,154,0.678337,"e) Módulo de Operación y Explotación, que perm...",NaN,NaN,NaN,Technical_assistance,NaN,NaN,...,0,0,0,1,0,0,1,0,7cffd6b,159


In [72]:
#groupby by tag
df_grouped_inst=df_doc.groupby('Tag').count()['doc_ref']
df_grouped_inst

Tag
Credit                  541
Direct_payment          512
Fine                    691
Supplies                447
Tax_deduction           450
Technical_assistance    448
Name: doc_ref, dtype: int64

In [73]:
df_grouped_inst.to_csv(r'data\inst_by_tag.csv')

In [74]:
#groupby by tag
df_grouped=df_doc.groupby('Tag').sum()[["Incentive_tag_any","Policy_tag_any","Policy_tag_majority","Incentive_uncategorised"]]
df_grouped


,Incentive_tag_any,Policy_tag_any,Policy_tag_majority,Incentive_uncategorised
Tag,,,,
Credit,302,203,191,99
Direct_payment,333,193,178,140
Fine,424,402,362,22
Supplies,147,32,24,115
Tax_deduction,186,39,31,147
Technical_assistance,94,84,68,10


In [75]:
df_grouped.to_csv(r'data\docs_by_tag.csv')

In [76]:
df_doc['doc_ref'].value_counts()



c7255b0    115
e7414a8     99
2bb3805     71
1739c19     57
3434b51     42
          ... 
e24a9dd      1
1edabde      1
3148bfd      1
cf1d4da      1
680d2f7      1
Name: doc_ref, Length: 633, dtype: int64

In [77]:
#groupby by tag
df_grouped_doc=df_doc.groupby('Tag').doc_ref.nunique()
df_grouped_doc

Tag
Credit                  189
Direct_payment          207
Fine                    252
Supplies                189
Tax_deduction           207
Technical_assistance    198
Name: doc_ref, dtype: int64

In [78]:
df_grouped_doc.to_csv(r'data\unique_docs_by_tag.csv')

In [79]:
#unique docs overall
df_udoc=df_doc.doc_ref.nunique()
df_udoc

633

In [80]:
#instances overall
df_inst=df_doc.count()['doc_ref']
df_inst

3089

In [81]:
##groupby by tag
df_summary=df_doc.sum()[["Incentive_tag_any","Policy_tag_any","Policy_tag_majority","Incentive_uncategorised"]]
df_summary


Incentive_tag_any          1486
Policy_tag_any              953
Policy_tag_majority         854
Incentive_uncategorised     533
dtype: object

In [82]:
# summarise by unique docs -
df_any_inc_udoc=df_doc.groupby(by="Incentive_tag_any").doc_ref.nunique()
print("Unique_docs_Incentive_tag_any")
print(df_any_inc_udoc)
df_any_tags_udoc=df_doc.groupby(by="Policy_tag_any").doc_ref.nunique()
print("Unique_docs_policy_tag_any")
print(df_any_tags_udoc)
df_maj_tags_udoc=df_doc.groupby(by="Policy_tag_majority").doc_ref.nunique()
print("Unique_docs_policy_tag_maj")
print(df_maj_tags_udoc)
df_inc_uncateg=df_doc.groupby(by="Incentive_uncategorised").doc_ref.nunique()
print("Unique_docs_incentive_uncategorised")
print(df_inc_uncateg)

Unique_docs_Incentive_tag_any
Incentive_tag_any
0    513
1    391
Name: doc_ref, dtype: int64
Unique_docs_policy_tag_any
Policy_tag_any
0    558
1    341
Name: doc_ref, dtype: int64
Unique_docs_policy_tag_maj
Policy_tag_majority
0    563
1    320
Name: doc_ref, dtype: int64
Unique_docs_incentive_uncategorised
Incentive_uncategorised
-1      5
 0    612
 1    195
Name: doc_ref, dtype: int64


In [83]:
# summarise by unique docs -
df_grouped_udoc=df_doc.groupby(by="Policy_tag_any").doc_ref.nunique()
df_grouped_udoc

Policy_tag_any
0    558
1    341
Name: doc_ref, dtype: int64

Now work on the inter-rater reliability - what is the similarity in tagging incentives. work where instances were tagged by all three raters

In [84]:
#get stats - how many shared sentences, how many each rater has added - start from the first dataset


df_grouped_raters=df_all_raters.groupby('Tag',).sum()[["Rater_1_policy_tag","Rater_2_policy_tag","Rater_3_policy_tag"]]
df_grouped_raters



,Rater_1_policy_tag,Rater_2_policy_tag,Rater_3_policy_tag
Tag,,,
Credit,54.0,164.0,135.0
Direct_payment,16.0,167.0,136.0
Fine,30.0,196.0,268.0
Supplies,15.0,21.0,17.0
Tax_deduction,14.0,9.0,29.0
Technical_assistance,21.0,12.0,69.0


In [85]:
df_grouped_raters=df_grouped_raters.transpose()
df_grouped_raters

Tag,Credit,Direct_payment,Fine,Supplies,Tax_deduction,Technical_assistance
Rater_1_policy_tag,54.0,16.0,30.0,15.0,14.0,21.0
Rater_2_policy_tag,164.0,167.0,196.0,21.0,9.0,12.0
Rater_3_policy_tag,135.0,136.0,268.0,17.0,29.0,69.0


In [86]:
df_grouped_raters.to_csv(r'data\tags_by_raters.csv')

In [87]:
# next - shared sentences across tags, shared documents across tags, average instances per doc, max instances per doc

#tags per doc

tag_by_doc=df_doc.groupby('doc_ref').Tag.count()
tag_by_doc

tag_by_doc.to_csv(r'data\tags_by_doc.csv')

In [88]:
# next - shared sentences across tags, shared documents across tags, average instances per doc, max instances per doc

#groupby by tag
uni_tag_by_doc=df_doc.groupby('doc_ref').Tag.nunique()
uni_tags_df=uni_tag_by_doc.to_frame().reset_index()

sum_uniq_tag=uni_tags_df.groupby('Tag').doc_ref.nunique()

uni_tags_df
#uni_tags_df.columns
sum_uniq_tag

uni_tag_by_doc.to_csv(r'data\uni_tags_by_doc.csv')


sum_uniq_tag.to_csv(r'data\sum_unitags_by_doc.csv')

In [89]:
#tags per sentence

tag_by_sent=df_doc.groupby('index').Tag.nunique()
tag_by_sent_df=tag_by_sent.to_frame().reset_index()
tag_by_sent_df

sum_uniq_tag_sent=tag_by_sent_df.groupby('Tag').index.nunique()
sum_uniq_tag_sent

#tag_by_sent.to_csv(r'data\tags_by_sentence.csv')

Tag
1    2274
2     272
3      53
4      28
Name: index, dtype: int64

In [90]:
#start with df_all_raters
#removed NaN rows - drop the alternative instrument label and set treshold
df_inter=df_all_raters.drop(["Rater_1_other_instrument","Rater_2_other_instrument", "Rater_3_other_instrument"], axis = 1)
#df_inter=df_inter.dropna(axis=0, thresh=2, subset=None, inplace=True)
#dropna(axis=0, thresh=5, subset=None, inplace=True)
df_inter

,Rater_1_original_rank,Rater_1_similarity_score,Rater_1_sentence,Rater_1_policy_tag,Rater_1_incentive_tag,Tag,Rater_2_policy_tag,Rater_2_incentive_tag,Rater_3_policy_tag,Rater_3_incentive_tag,...,Pos_policy_tags,Pos_incentive_tags,Alternative_tags,Incentive_tag_majority,Incentive_tag_any,No_incentive_tag,Policy_tag_majority,Policy_tag_any,No_policy_tag,Incentive_uncategorised
43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Credit,0.0,1.0,0.0,1.0,...,0.0,3.0,3,1,1,0,0,0,1,1
12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,Credit,0.0,0.0,0.0,1.0,...,0.0,1.0,0,0,1,0,0,0,1,1
26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Credit,0.0,1.0,0.0,1.0,...,0.0,3.0,3,1,1,0,0,0,1,1
27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,Credit,0.0,0.0,0.0,1.0,...,0.0,1.0,0,0,1,0,0,0,1,1
15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,Credit,1.0,1.0,1.0,1.0,...,3.0,3.0,0,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f1ebd15_30,26,0.678436,"Para estos efectos, ""la Corporación"" deberá fo...",NaN,NaN,Technical_assistance,NaN,NaN,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0
daebb8c_7,152,0.678430,Que las acciones y obras que componen el refer...,NaN,NaN,Technical_assistance,NaN,NaN,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0
2bb3805_424,153,0.678387,En estos casos corresponderá al SERVIU el desa...,NaN,NaN,Technical_assistance,NaN,NaN,1.0,1.0,...,1.0,1.0,0,1,1,0,1,1,0,0
7cffd6b_159,154,0.678337,"e) Módulo de Operación y Explotación, que perm...",NaN,NaN,Technical_assistance,NaN,NaN,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0


In [91]:
df_inter_trim=df_inter.dropna(axis=0)
df_inter_trim

,Rater_1_original_rank,Rater_1_similarity_score,Rater_1_sentence,Rater_1_policy_tag,Rater_1_incentive_tag,Tag,Rater_2_policy_tag,Rater_2_incentive_tag,Rater_3_policy_tag,Rater_3_incentive_tag,...,Pos_policy_tags,Pos_incentive_tags,Alternative_tags,Incentive_tag_majority,Incentive_tag_any,No_incentive_tag,Policy_tag_majority,Policy_tag_any,No_policy_tag,Incentive_uncategorised
43f0373_23,1,0.764215,Apoyar tecnicamente la ejecucion de programas ...,0.0,1.0,Credit,0.0,1.0,0.0,1.0,...,0.0,3.0,3,1,1,0,0,0,1,1
12a91a7_159,2,0.733801,"- Focalizar, en mejor forma, los incentivos es...",0.0,0.0,Credit,0.0,0.0,0.0,1.0,...,0.0,1.0,0,0,1,0,0,0,1,1
26b00aa_103,3,0.725276,"1,1 Propiciar el desarrollo de los asentamient...",0.0,1.0,Credit,0.0,1.0,0.0,1.0,...,0.0,3.0,3,1,1,0,0,0,1,1
27d89cb_179,4,0.720910,Destinados a cubrir las necesidades productiva...,0.0,0.0,Credit,0.0,0.0,0.0,1.0,...,0.0,1.0,0,0,1,0,0,0,1,1
15df6c5_144,1,0.717441,- El Servicio podrá conceder los siguientes pr...,1.0,1.0,Credit,1.0,1.0,1.0,1.0,...,3.0,3.0,0,1,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61c0077_58,63,0.760798,- Características técnicas y de operación de l...,0.0,0.0,Technical_assistance,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0
731dbf0_34,116,0.760442,Los aspectos tecnicos y administrativos relaci...,0.0,0.0,Technical_assistance,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0
e089569_45,64,0.759861,Los Estudios de Interconexión deberán ser elab...,0.0,0.0,Technical_assistance,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0
9b07f76_54,119,0.759089,U otros materiales que garanticen el aislamien...,0.0,0.0,Technical_assistance,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0


In [92]:
coder1=df_inter_trim["Rater_1_incentive_tag"].to_list()
coder2=df_inter_trim["Rater_2_incentive_tag"].to_list()
coder3=df_inter_trim["Rater_3_incentive_tag"].to_list()


In [93]:
#GN now apply inter-rater reliability  - check whether merge with the sentences are needed, only by index is sufficient

#try with a list rather than dimensions, like coder1 = [1,0,2,0,1,1,2,0,1,1]

  
formatted_codes = [[1,i,coder1[i]] for i in range(len(coder1))] + [[2,i,coder2[i]] for i in range(len(coder2))]  + [[3,i,coder3[i]] for i in range(len(coder3))]
    
ratingtask = agreement.AnnotationTask(data=formatted_codes)
print(' Fleiss\'s Kappa:',ratingtask.multi_kappa())

 Fleiss's Kappa: 0.4950915616545709


In [94]:
coder1_tag=df_inter_trim["Rater_1_policy_tag"].to_list()
coder2_tag=df_inter_trim["Rater_2_policy_tag"].to_list()
coder3_tag=df_inter_trim["Rater_3_policy_tag"].to_list()


In [95]:
#GN now apply inter-rater reliability  - check whether merge with the sentences are needed, only by index is sufficient


  
formatted_codes = [[1,i,coder1_tag[i]] for i in range(len(coder1_tag))] + [[2,i,coder2_tag[i]] for i in range(len(coder2_tag))]  + [[3,i,coder3_tag[i]] for i in range(len(coder3_tag))]
    
ratingtask = agreement.AnnotationTask(data=formatted_codes)
print(' Fleiss\'s Kappa:',ratingtask.multi_kappa())

 Fleiss's Kappa: 0.5627594715810887
